# Introducción a la Ciencia de Datos con Python - Reporte Final

## Título
Tiempo de atención en sucursales de entidad bancaria local

## Integrantes
- Damián Augusto Meza Candia
- Javier Augusto Meza Candia

## Contenido
- [Introducción](#Introducción)
- [Descripción del dataset](#Descripción-del-dataset)
- [Tipo de problema planteado](#Tipo-de-problema-planteado)
- [Metodología empleada para resolver el problema](#Metodología-empleada-para-resolver-el-problema)
    - [Limpieza de datos](#Limpieza-de-datos)
    - [Análisis de datos](#Análisis-de-datos)
- [Métricas de desempeño utilizadas](#Métricas-de-desempeño-utilizadas)
- [Descripción de los resultados obtenidos](#Descripción-de-los-resultados-obtenidos)
- [Referencias](#Referencias)

## Introducción
El problema que queremos resolver se trata de predecir el tiempo de espera y atención de los clientes en las diferentes sucursales de una entidad bancaria local, según el tipo de operación/servicio que desean realizar, de manera a optimizar los recursos y lograr una atención más eficaz dentro de las mismas.

Preguntas claves:
- ¿El tiempo de espera está directamente relacionado con el tipo de operación a realizar?
- ¿El tiempo de espera está relacionado con la variable del día de la semana? ¿O el horario del día?
- ¿El tiempo de espera está relacionado con la sucursal de atención?
- ¿El tiempo de espera está relacionado con la categoría de cliente?

## Descripción del dataset
El dataset contiene datos de tickets de atención a clientes presenciales en sucursales de una entidad bancaria local durante el mes de Octubre de 2022.
Sobre cada atención en particular se cuenta con los siguientes datos: sucursal, zona de la sucursal, sector dentro de la sucursal, box de atención, tipo de atención, estado de atención, categoría del cliente, identificación del cliente, usuario encargado de la atención, tiempo de espera, tiempo de atención, motivo de cierre del ticket, entre otros.

- **Variables:** Código de sucursal, tipo de cola, nro. de atención general, fecha de ingreso a la sucursal, fecha y hora de inicio del llamado, fecha y hora de inicio de atención, fecha y hora de fin de atención, código de caja de atención, nro. de ticket, categoría de cliente, tiempo de espera, tiempo atendido, entre otros.
- **Tipos de datos:** Contamos con datos categóricos, datos de fechas y datos numéricos.
- **Número de registros:** 69.481 registros
- **Datos faltantes:** contamos con datos faltantes en la columna de fecha y hora de inicio de atención o fin de atención, en cuyo caso optamos por ignorar los registros (2.450 registros). También descartamos las columnas sin dato alguno en ninguna fila.

## Tipo de problema planteado
Identificamos que el problema corresponde a un problema de regresión.

## Metodología empleada para resolver el problema
Llevaremos a cabo mediante el lenguaje de programación Python y las librerías disponibles, los pasos necesarios para extraer la información de los sistemas transaccionales, comprender la situación actual y descubrir patrones que ayuden a la toma de decisiones para mejorar la calidad de la atención a los clientes.

### Limpieza de datos

In [ ]:
import numpy as np
import pandas as pd

# read CSV file from the 'data' subdirectory using a relative path
data = pd.read_csv('dataset_ticket_hist.csv', index_col=0, low_memory=False)

In [ ]:
# display the first 5 rows
data.head()

In [ ]:
# display the last 5 rows
data.tail()

In [ ]:
# check the shape of the DataFrame (rows, columns)
data.shape

In [ ]:
# eliminamos las columnas que no se necesitan
data.drop('COLHORING', inplace=True, axis=1)
data.drop('COLABN', inplace=True, axis=1)
data.drop('DESPLAN_CONSU', inplace=True, axis=1)
data.drop('MODOPAGO', inplace=True, axis=1)
data.drop('TIPCUENTA', inplace=True, axis=1)
data.drop('ESTCUENTA', inplace=True, axis=1)
data.drop('DESMARMOD', inplace=True, axis=1)
data.drop('SITUEQUIP', inplace=True, axis=1)
data.drop('MOTREGIS', inplace=True, axis=1)
data.drop('MHORAENV', inplace=True, axis=1)
data.drop('MHORACONF', inplace=True, axis=1)
data.drop('EST_PORT', inplace=True, axis=1)

# eliminamos otras columnas que no se necesitan
data.drop('NRO_DE_ATENCION_GENERAL', inplace=True, axis=1)
data.drop('USRCOD', inplace=True, axis=1)
data.drop('BOXCOD', inplace=True, axis=1)
data.drop('COLNROTICK', inplace=True, axis=1)
data.drop('TICKEST', inplace=True, axis=1)
data.drop('USRATENDIO', inplace=True, axis=1)
data.drop('CONTATEN', inplace=True, axis=1)
data.drop('tiempo2daLinea', inplace=True, axis=1)
data.drop('yallamo', inplace=True, axis=1)
data.drop('derivado_a', inplace=True, axis=1)
data.drop('ticket_inicial', inplace=True, axis=1)
data.drop('enviado_a_standby', inplace=True, axis=1)
data.drop('COLHORAINITSTANBY', inplace=True, axis=1)
data.drop('SERV_INICIAL', inplace=True, axis=1)

In [ ]:
# eliminamos las filas con datos faltantes
data = data.dropna(subset=['COLHORAINIAT'])
data = data.dropna(subset=['COLHORAFINAT'])

In [ ]:
# check the shape of the DataFrame (rows, columns)
data.shape

La función *apply()* en DataFrame tomará alguna función arbitraria que haya escrito y la aplicará a una *serie* (una sola columna) o DataFrame en todas las filas o columnas.

In [ ]:
def todatetime(row):
    #La fila es un único objeto de *Series* que es una sola fila indexada por valores de columna.
    #Convertimos el tipo de dato en una nueva entrada en la serie.
    row['FEC_INGRESO']=pd.to_datetime(row['COLFECING'], format="%d/%m/%Y %H:%M:%S")
    row['FEC_INI_ATENCION']=pd.to_datetime(row['COLHORAINIAT'], format="%d/%m/%Y %H:%M:%S")
    row['FEC_FIN_ATENCION']=pd.to_datetime(row['COLHORAFINAT'], format="%d/%m/%Y %H:%M:%S")
    row['FEC_INI_LLAMADA']=pd.to_datetime(row['COLHORAINILLA'], format="%d/%m/%Y %H:%M:%S")
    #Ahora solo retornamos la fila y la función apply() se encargará de fusionarlas de vuelta en un DataFrame
    return row

In [ ]:
data=data.apply(todatetime, axis="columns")
data.head()

In [ ]:
# eliminamos las viejas columnas que YA no se necesitan
data.drop('COLFECING', inplace=True, axis=1)
data.drop('COLHORAINIAT', inplace=True, axis=1)
data.drop('COLHORAFINAT', inplace=True, axis=1)
data.drop('COLHORAINILLA', inplace=True, axis=1)
data.head()

In [ ]:
data['TIEMPO_ESPERA'] = round((data.FEC_INI_ATENCION-data.FEC_INGRESO) / pd.Timedelta(minutes=1), 2)
data['TIEMPO_TOTAL_ATENCION'] = round((data.FEC_FIN_ATENCION-data.FEC_INI_ATENCION) / pd.Timedelta(minutes=1), 2)
data['TIPO_OPERACION'] = data.TIPCOLACOD.str.slice(3)
# Find unique values of a column
print(data['TIPO_OPERACION'].unique())
data.head()

In [ ]:
def toweekday(row):
    #La fila es un único objeto de *Series* que es una sola fila indexada por valores de columna.
    #Convertimos el tipo de dato en una nueva entrada en la serie.
    row['DIA_SEMANA']=row['FEC_INGRESO'].isoweekday()
    row['HORA_DIA']=row['FEC_INGRESO'].hour
    row['DIA_MES']=row['FEC_INGRESO'].day
    #Ahora solo retornamos la fila y la función apply() se encargará de fusionarlas de vuelta en un DataFrame
    return row

In [ ]:
#Categorizamos por dia de la semana
data=data.apply(toweekday, axis="columns")
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

### Análisis de datos

Analizamos las funciones de agregación del tiempo de espera (en minutos):

In [ ]:
# Por día de la semana
# 1=Lunes, 2=Martes, 3=Miércoles, 4=Jueves, 5=Viernes
data.groupby("DIA_SEMANA").agg({"TIEMPO_ESPERA":(np.min,np.max,np.average,np.median,np.mean,np.std)})

In [ ]:
# Por hora del día
data.groupby("HORA_DIA").agg({"TIEMPO_ESPERA":(np.min,np.max,np.average,np.median,np.mean,np.std)})

#### Recorte
El recorte implica la limitación de todos los valores por debajo o por encima de un determinado valor. El recorte es útil cuando una columna contiene algunos valores atípicos. Podemos establecer un valor máximo vmax y un valor mínimo vmin y establecer todos los valores atípicos mayores que el valor máximo en vmax y todos los valores atípicos menores que el valor mínimo en vmin.
Establecemos los valores mínimo y máximo para el tiempo de espera y el tiempo total de atención (en minutos):
- vmin = 0
- vmax = 90

In [ ]:
vmin = 0
vmax = 90

data['TIEMPO_ESPERA_CLIP'] = data['TIEMPO_ESPERA'].apply(lambda x: vmax if x > vmax else vmin if x < vmin else x)
data['TIEMPO_TOTAL_ATENCION_CLIP'] = data['TIEMPO_TOTAL_ATENCION'].apply(lambda x: vmax if x > vmax else vmin if x < vmin else x)
data.head()

Volvemos a analizar las funciones de agregación del tiempo de espera (en minutos):

In [ ]:
# Por día de la semana
# 1=Lunes, 2=Martes, 3=Miércoles, 4=Jueves, 5=Viernes
data.groupby("DIA_SEMANA").agg({"TIEMPO_ESPERA_CLIP":(np.min,np.max,np.average,np.median,np.mean,np.std)})

In [ ]:
# Por hora del día
data.groupby("HORA_DIA").agg({"TIEMPO_ESPERA_CLIP":(np.min,np.max,np.average,np.median,np.mean,np.std)})

#### Visualización

In [ ]:
# conventional way to import seaborn
import seaborn as sns

# allow plots to appear within the notebook
%matplotlib inline

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.pairplot(data, x_vars=['DIA_SEMANA','DIA_MES','HORA_DIA'], y_vars='TIEMPO_ESPERA_CLIP', height=8, aspect=0.6, kind='reg')
sns.pairplot(data, x_vars=['DIA_SEMANA','DIA_MES','HORA_DIA'], y_vars='TIEMPO_TOTAL_ATENCION_CLIP', height=8, aspect=0.6, kind='hist')

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.barplot(data, x='CATCLI', y='TIEMPO_ESPERA_CLIP')

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.barplot(data, x='CATCLI', y='TIEMPO_TOTAL_ATENCION_CLIP')

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.barplot(data, x='TIEMPO_ESPERA_CLIP', y='SUCCOD', orient='h')

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.barplot(data, x='TIEMPO_TOTAL_ATENCION_CLIP', y='SUCCOD', orient='h')

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.pairplot(data, x_vars=['TIPO_OPERACION'], y_vars='TIEMPO_ESPERA_CLIP', height=7, aspect=2, kind='scatter')
sns.pairplot(data, x_vars=['TIPO_OPERACION'], y_vars='TIEMPO_TOTAL_ATENCION_CLIP', height=7, aspect=2, kind='scatter')

#### Codificación de datos categóricos
Muchos algoritmos de aprendizaje automático son incapaces de procesar variables categóricas. Por ejemplo: Bajo, Medio, Alto.

Por lo tanto, es importante codificar los datos en una forma adecuada para poder preprocesar estas variables. La codificación consiste en convertir todas las variables de entrada y salida en numéricas. De este modo, el modelo podrá comprender y extraer la información generando la salida deseada. Los datos categóricos varían en función del número de valores posibles.

In [ ]:
# create new dataframe for the categorical variable: CATCLI
catcli_dummies = pd.get_dummies(data.CATCLI)
data = pd.concat([data, catcli_dummies], axis=1)
data.drop('HH', inplace=True, axis=1)
data.head()

In [ ]:
# create a Python list of feature names
feature_cols = ['AA','KK','LL','MM','N','NN','PP','QQ','RR','DIA_MES','DIA_SEMANA','HORA_DIA']

# use the list to select a subset of the original DataFrame
X = data[feature_cols]

# print the first 5 and last 5 rows
X

In [ ]:
# check the type and shape of X
print(type(X))
print(X.shape)

In [ ]:
# select a Series from the DataFrame
y = data['TIEMPO_ESPERA_CLIP']

# print the first 5 values
y.head()

In [ ]:
# check the type and shape of y
print(type(y))
print(y.shape)

#### Separando conjunto de entrenamiento
En scikit-learn una división aleatoria en conjuntos de entrenamiento y de prueba puede ser rápidamente calculada con la función de ayuda *train_test_split*.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

In [ ]:
# default split is 75% for training and 25% for testing
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

#### Regresión lineal en scikit-learn
LinearRegression ajusta un modelo lineal con coeficientes w = (w1, ..., wp) para minimizar la suma residual de cuadrados entre los objetivos observados en el conjunto de datos y los objetivos predichos por la aproximación lineal.

In [ ]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

#### Interpretando los coeficientes

In [ ]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

In [ ]:
# pair the feature names with the coefficients
list(zip(feature_cols, linreg.coef_))

#### Haciendo predicciones

In [ ]:
# make predictions on the testing set
y_pred = linreg.predict(X_test)

## Métricas de desempeño utilizadas

### Métricas de evaluación del modelo para la regresión
Las métricas de evaluación para problemas de clasificación, como **precisión**, no son útiles para problemas de regresión. En cambio, necesitamos métricas de evaluación diseñadas para comparar valores continuos.

Vamos a crear algunas predicciones numéricas de ejemplo y calcular **tres métricas de evaluación comunes** para problemas de regresión:

In [ ]:
from sklearn import metrics

# calculate MAE using scikit-learn
print(f"MAE: {metrics.mean_absolute_error(y_test, y_pred)}")

# calculate MSE using scikit-learn
print(f"MSE: {metrics.mean_squared_error(y_test, y_pred)}")

# calculate RMSE using scikit-learn
print(f"RMSE: {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}")

### KFold
El método Kfold devuelve el orden de las muestras elegidas para los conjuntos de entrenamiento y test en cada pliegue. En un marco de datos pandas tenemos que usar la función *.iloc* para obtener las filas correctas.

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
i = 1

print(f"************************************************************************")
for train_index, test_index in kf.split(X):
    cv_X_train = X.iloc[train_index]
    cv_X_test = X.iloc[test_index]
    cv_y_train = y.iloc[train_index]
    cv_y_test = y.iloc[test_index]
    
    #Train the model
    linreg.fit(cv_X_train, cv_y_train) #Training the model
    cv_y_pred = linreg.predict(cv_X_test)
    print(f"MAE for the fold no. {i} on the test set: {metrics.mean_absolute_error(cv_y_test, cv_y_pred)}")
    # calculate MSE using scikit-learn
    print(f"MSE for the fold no. {i} on the test set: {metrics.mean_squared_error(cv_y_test, cv_y_pred)}")
    # calculate RMSE using scikit-learn
    print(f"RMSE for the fold no. {i} on the test set: {np.sqrt(metrics.mean_squared_error(cv_y_test, cv_y_pred))}")
    print(f"************************************************************************")

    i += 1

## Descripción de los resultados obtenidos
_TODO_

## Referencias
- [Gomes, Dipta & Nabil, Rashidul & Nur, Kamruddin. (2020). Banking Queue Waiting Time Prediction based on Predicted Service Time using Support Vector Regression. 10.1109/ICCAKM46823.2020.9051490.](https://www.researchgate.net/publication/339228555_Banking_Queue_Waiting_Time_Prediction_based_on_Predicted_Service_Time_using_Support_Vector_Regression)